# 11/14/2016: Session 8
# Internet and Persistence

In [ ]:
# flask is like Django-lite on the web

In [ ]:
from flask import Flask
my_app = Flask(__name__)      # a Flask object

@my_app.route('/this')
def dothis():
    return 'Get thee to a nunnery!<BR><BR>Say <A HREF="/">Hi!</A>'


@my_app.route('/')            # called when visiting web URL 127.0.0.1:5000/
def hello_world():
    print '*** This String Tho ***' # print statements get printed to log, not webpage, serves debugging purposes
    return 'Hello World!<BR><BR>Say <A HREF="/bye">Bye!</A>'

@my_app.route('/bye')         # called when visiting web URL 127.0.0.1:5000/bye
def goodbye():
    return dothis()

if __name__ == '__main__':
    my_app.run(debug=True, port=5000)    # app starts serving in debug mode on port 5000

In [ ]:
# reading from form submission
import flask
my_app = flask.Flask(__name__)      # a Flask object

@my_app.route('/')            # called when visiting web URL 127.0.0.1:5000/
def hello_world():
    return 'What is your <A HREF="/bye"> name</A>'

@my_app.route('/bye')         # called when visiting web URL 127.0.0.1:5000/bye
def goodbye():
    fname = flask.request.args.get('fname')              # like dict.get()
    lname = flask.request.values['lname']                # like dict access
    return 'fname: {}, lname: {}'.format(fname, lname)

if __name__ == '__main__':
    my_app.run(debug=True, port=5000)    # app starts serving in debug mode on port 5000

<!DOCTYPE html>
<html lang="en">
  <body>
    <form action = "http://127.0.0.1:5000/bye">
      First name: <input name="fname"><br>
      Last name: <input name="lname"><br>
      <input type="submit" value="Go!">
    </form>
  </body>
</html>

## JSON

In [1]:
j = {
   "key1":  ["a", "b", "c"],
   "key2":  {
              "innerkey1": 5,
              "innerkey2": "woah"
            },
   "key3":  55.09,
   "key4":  "hello"
}

In [5]:
fh = open('mystruct.json')    # file contains JSON
mys = json.load(fh)           # load from a file
fh.close()

print mys['key2']['innerkey2']   # woah

{'key3': 55.09, 'key2': {'innerkey2': 'woah', 'innerkey1': 5}, 'key1': ['a', 'b', 'c'], 'key4': 'hello'}


easy to make mistakes when coding json files by hand - very particular about formatting
instead, use python to code json files and read them

In [12]:
import pickle
import datetime
x = {'a':1, 'b':2, 'c':[3,4,5], 'd':datetime.date(2016,11,26)}
print x['d']

2016-11-26


### pickle for object serialization
Any Python data object can be written to disk without modification.  

In [19]:
import pickle
import datetime
br = [('joe', 22, 'clerk', 33000.0), ('pete', 34, 'salesman', 51000.0), datetime.date(2016,5,31)]
F = open('emptyfile.pickle', 'w')
pickle.dump(br, F)
F.close()

## later
G = open('emptyfile.pickle')
J = pickle.load(G)
print J
print J[2]
    #  [('joe', 22, 'clerk', 33000.0), ('pete', 34, 'salesman', 51000.0), 

[('joe', 22, 'clerk', 33000.0), ('pete', 34, 'salesman', 51000.0), datetime.date(2016, 5, 31)]
2016-05-31


For ease of use, custom-designed objects are superior.  As they can behave like Python objects, they can be used in a variety of familiar situations.  As they are custom-designed, they can be made to hold data in any form.  

Since custom-designed objects can be serialized using pickle, they can be stored directly without using object<->relational or object<->JSON translations.  pickle is slower than RDBS and JSON, but it is a convenient option for lightweight applications.  

Consider this pair of data objects:  one holds items and the other is an item:

In [47]:
import pickle

class ItemList(object):

    def __init__(self, name):
        self.name = name
        self.items = []
        self.index = -1

    def add(self, text):
        self.items.append(Item(text))

    def __iter__(self):
        return self

    def next(self):
        self.index += 1
        if self.index > len(self.items) - 1:
            raise StopIteration
        return self.items[self.index]

class Item(object):
    def __init__(self, text):
        self.text = text

In [48]:
this_list = ItemList('mylist')

this_list.add('line A')
this_list.add('line B')
this_list.add('line C')

In [32]:
wfh = open('picklefile.pickle', 'w')
pickle.dump(this_list, wfh)
wfh.close()

### THE NEXT DAY...

fh = open('picklefile.pickle')

resurrected_list = pickle.load(fh)

print 'items for {}'.format(resurrected_list.name)
for i, item in enumerate(resurrected_list):
    print ' {}. '.format(i+1) + item.text

items for mylist
 1. line A
 2. line B
 3. line C


In [52]:
print this_list.next()

In [53]:
# SQLite
def insert_db(fname, lname):
    sql = INSERT_QUERY.format(TABLE_NAME, fname, lname)
    execute_query(sql)

def get_db_data():
    sql = SELECT_QUERY.format(TABLE_NAME)
    c = execute_query(sql)    
    return c.fetchall()

In [ ]:
def execute_query(sql):
    conn = get_db()
    c = conn.cursor()
    c.execute(sql)
    conn.commit()
    return c

def get_db():
    db = getattr(flask.g, '_database', None)
    if db is None:
        db = flask.g._database = sqlite3.connect(DATABASE)
    return db
    
@app.teardown_appcontext
def close_connection(exception):
    db = getattr(flask.g, '_database', None)
    if db is not None:
        db.close()